In [1]:
import pandas as pd
import pymorphy2 as pm
morph = pm.MorphAnalyzer()
import ast
from tqdm import tqdm_notebook

import random
from random import shuffle

import matplotlib.pyplot as plt
plt.style.use('bmh')
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import numpy as np
#import xgboost as xgb

In [2]:
matplotlib inline

In [90]:
dftrain, dftest = pd.read_csv("./train_tokens_facts_number.csv"), pd.read_csv("./test_tokens_facts_number.csv")

In [91]:
test_question = dftest.groupby(['question']).agg({'paragraph_id': 'count'})
test_question = pd.DataFrame({'question': test_question.index, 'count': test_question.paragraph_id})
test_question['count'] = test_question['count'].astype(int)
dftest = dftest.merge(test_question, how = 'left', on='question')

In [92]:
train_question = dftrain.groupby(['question']).agg({'paragraph_id': 'count'})
train_question = pd.DataFrame({'question': train_question.index, 'count': train_question.paragraph_id})
train_question['count'] = train_question['count'].astype(int)
dftrain = dftrain.merge(train_question, how = 'left', on='question')

In [93]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import SnowballStemmer 
from nltk.corpus import stopwords

import string

In [94]:
from sklearn.metrics.pairwise import paired_cosine_distances

In [97]:
questions_train = dftrain[dftrain['count']>1].question_id.unique()

In [98]:
questions_test = dftest[dftest['count']>1].question_id.unique()

In [ ]:
for name, qlist, df in [('train', questions_train, dftrain[dftrain['count']>1]), ('test', questions_test, dftest[dftest['count']>1])]: 
    for i in tqdm_notebook(qlist, total=len(qlist), desc= name + ' questions'):
        texts = list(df[df['question_id']==i]['tokens_paragraph'].unique())+\
        list(df[df['question_id']==i]['tokens_question'].unique())
        tfidf = TfidfVectorizer()
        tfidf_matrix=tfidf.fit_transform(texts)
        qvec = tfidf_matrix[len(texts)-1]
        for n, t in tqdm_notebook(enumerate(texts), total=len(texts), desc= name + ' paragraphs', leave=False):
            if n!=len(texts)-1:
                svec = tfidf_matrix[n]
                score=float(paired_cosine_distances(svec,qvec))
                df.loc[(df['tokens_paragraph']==t)&(df['question_id']==i),'score'] = score
            else:
                pass

Exception in thread Thread-11:
Traceback (most recent call last):
  File "C:\Users\anna.vozmishcheva\AppData\Local\Continuum\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\anna.vozmishcheva\AppData\Local\Continuum\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 106, in run
    if instance.miniters > 1 and \
AttributeError: 'tqdm_notebook' object has no attribute 'miniters'



In [14]:
dftest[dftest['count']==51].question_id.unique()

array([14349], dtype=int64)

In [15]:
#list(dftest[dftest['question_id']==14349]['tokens_paragraph'].unique())

In [101]:
texts = list(dftest[dftest['question_id']==14349]['tokens_paragraph'].unique())+list(dftest[dftest['question_id']==14349]['tokens_question'].unique())

In [102]:
len(texts)

52

In [30]:
#texts = list(dftest[dftest['question_id']==14349]['paragraph'].unique())+list(dftest[dftest['question_id']==14349]['question'].unique())

In [31]:
texts

["['помимо', 'британия', 'и', 'сша', 'свой', 'панк', 'сцена', 'формироваться', 'в', 'австралия', 'в', '1965', 'год', 'группа', 'los', 'saicos', 'записать', 'композиция', 'demolicion', 'стать', 'один', 'известный', 'образцов', 'ранний', 'панк', 'портал', 'allmusic', 'так', 'отзываться', 'о', 'это', 'коллектив', 'гитара', 'не', 'похожий', 'ни', 'на', 'что', 'и', 'звучать', 'как', 'фонтан', 'искра', 'ударный', 'пульсировать', 'в', 'дух', 'пост', 'сёрф', 'а', 'вокалист', 'не', 'попадать', 'в', 'нота', 'подытожить', 'этот', 'ребята', 'уже', 'быть', 'панк', 'группа', 'даже', 'если', 'никто', 'за', 'предел', 'лима', 'не', 'знать', 'о', 'это', 'в', 'то', 'время', 'в', 'свой', 'очередь', 'выпустить', 'в', 'тот', 'же', 'год', 'сингл', 'гаражный', 'рок', 'группа', 'the', 'missing', 'links', 'wild', 'about', 'you', '1965', 'стать', 'источник', 'вдохновение', 'для', 'один', 'из', 'самый', 'известный', 'коллектив', 'австралийский', 'панк', 'рок', 'the', 'saints', 'десятилетие', 'спустя']",
 "['тот',

In [45]:
tfidf = TfidfVectorizer()
tfidf_matrix=tfidf.fit_transform(texts)

In [46]:
tfidf_matrix.shape

(52, 2278)

In [47]:
print(tfidf_matrix[51])

  (0, 1530)	0.0721192707186
  (0, 1166)	0.0985545202554
  (0, 1328)	0.07781392132
  (0, 578)	0.0763441536404
  (0, 1850)	0.122696109915
  (0, 738)	0.160849435896
  (0, 999)	0.100612274535
  (0, 997)	0.14696423081
  (0, 2044)	0.151280033851
  (0, 1358)	0.193316187084
  (0, 841)	0.151280033851
  (0, 1623)	0.178394190107
  (0, 415)	0.239668143359
  (0, 101)	0.239668143359
  (0, 5)	0.239668143359
  (0, 266)	0.239668143359
  (0, 164)	0.212553987103
  (0, 1125)	0.239668143359
  (0, 814)	0.166202030829
  (0, 1317)	0.202245663314
  (0, 1169)	0.239668143359
  (0, 1451)	0.193316187084
  (0, 759)	0.224746146381
  (0, 1277)	0.239668143359
  (0, 712)	0.258905943378
  (0, 1036)	0.239668143359
  (0, 461)	0.239668143359


In [50]:
tfidf_matrix.shape

(52, 2278)

In [48]:
from sklearn.metrics.pairwise import paired_cosine_distances

In [1]:
qvec = tfidf_matrix[51]
for n,i in enumerate(texts):
    if n!=51:
        svec = tfidf_matrix[n]
        score=float(paired_cosine_distances(svec,qvec))
        print(score)
        dftest.loc[(dftest['tokens_paragraph']==i)&(dftest['question_id']==14349),'score'] = score
    else:
        pass

NameError: name 'tfidf_matrix' is not defined

In [82]:
dftest[(dftest['count']==51)&(dftest['score']<0.8)].paragraph.values[1]

'В 1980-м году Бретт Гуревич (Bad Religion) основал лейбл Epitaph Records, ставший стартовой площадкой для многих будущих поп-панк-групп. Некоторые из этих коллективов (The Queers и Screeching Weasel) впоследствии начали комбинировать панк с ещё более беззаботными поп-мелодиями, тем самым оказав мощнейшее влияние на будущих звёзд жанра — Rancid, Blink-182, Green Day и The Offspring — добившихся популярности среди мейнстримовой аудитории в середине 90-х и записавших ряд сверхуспешных дисков. Помимо музыкальной составляющей, некоторые представители движения начали акцентировать тексты своих песен на юмористическом и оскорбительном содержании, среди наиболее известных таких команд были — The Vandals и Guttermouth. По прошествии времени, помимо Калифорнии, крупные поп-панк-сцены сформировались в Чикаго и Миннеаполисе, некоторые коллективы которых, такие как Dillinger Four, запустили обратную тенденцию — вернувшись к более агрессивному, оригинальному звучанию в пределах жанра, а также сохра

In [61]:
dftest[dftest['count']==51].question.values[0]

'Какие коллективы впоследствии начали комбинировать панк с ещё более беззаботными поп-мелодиями, тем самым оказав мощнейшее влияние на будущих звёзд жанра — Rancid, Blink-182, Green Day и The Offspring?'